In [1]:
import sys 
#sys.path.append("codes/modules") # add custom Vibe 's modules
sys.path.append('../..') # add standard 's modules
sys.path.append('../modules')

import pyfesom as pf
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
import numpy as np
from netCDF4 import Dataset
import os
import time
import pandas as pd
import xarray as xr
sys.path.append('/home/hbkoziel/pyfesom/pyfesom/cmocean/')
import cmocean as cmap

No joblib
no cmocean


In [3]:
# Loading mesh for run

mesh_id    = 'meshArc4.5'
meshpath   = '/scratch/usr/hbkoziel/mesh/'+mesh_id+'/'              # Defining path where mesh is stored
mesh = pf.load_mesh(meshpath, usepickle=True, get3d=True)                                    # Loading mesh, stores it in mesh.****  
#mesh = pf.fesom_mesh(meshpath, get3d=True)
#mesh.zlevs = -mesh.zlevs                                            # Depth is made negative

tracername = 'LIM'
first_year = 1985
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
runid	= 'Arc12'

data_path  = '/scratch/usr/hbkoziel/'+runid+'/netcdf/'

/scratch/usr/hbkoziel/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /scratch/usr/hbkoziel/mesh/meshArc4.5/pickle_mesh


In [4]:
dates = pd.date_range('1985', freq="M", periods= 12*len(np.arange(1985,2016,1)))
data = xr.open_mfdataset(data_path+tracername+'*.nc', concat_dim='time')
data['time'] = dates
#data.time.data = dates

IOError: no files to open

In [ ]:
data

In [ ]:
data2 = data.sel(time=data.time.dt.month.isin([7, 8, 9]))
data2 = data2.compute()
data2

data2 = data2.resample(time='YS').mean(dim='time')
data2 = data2.compute()
data2

In [ ]:
data_np = data2.Nlimphy.sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
data_np = data_np.compute()

data_nd = data2.Nlimdia.sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
data_nd = data_nd.compute()

data_si = data2.Silim.sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
data_si = data_si.compute()

data_lp = data2.Phy_Light_limiter.sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
data_lp = data_lp.compute()

data_ld = data2.Dia_Light_limiter.sel(time=slice(str(first_year)+'-01-01', str(last_year)+'-12-31'))
data_ld = data_ld.compute()

In [ ]:
ncfile      = meshpath+'Arc4.5.initial.mesh.diag.nc'
f           = Dataset(ncfile, 'r')
NodalAreaModel = f.variables['cluster_area'][:]

In [ ]:
data_lim_sertemp = 100 - 100*(data_np + data_nd)/2 #+ data_si_mean
data_lim_mean = data_lim_sertemp.mean(dim='time')
pff = np.polyfit(years, data_lim_sertemp, 1)
data_lim_trend = pff[0]

In [ ]:
print data_lim_mean.shape
print data_lim_trend.shape
print data_lim_sertemp.shape

In [ ]:
fig = plt.figure(figsize=(15, 10), facecolor='w', edgecolor='k')

#MLD_sept_mean[MLD_sept_mean == 0] = np.nan
data, elem_no_nan = pf.get_data(data_lim_mean,mesh,0)
ax1      = fig.add_subplot(121)
contours = np.arange(0, 101, 10)
m = Basemap(projection='nplaea',boundinglat=70,lon_0=0,resolution='l')
x, y = m(mesh.x2, mesh.y2)
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
m.drawmapboundary(fill_color='0.7')
m.fillcontinents(color='.5',lake_color='.7')
im=plt.tricontourf(x, y, elem_no_nan, (data) , levels=contours, cmap=cmap.cm.thermal, extend='both')
label = 'DIN limitation occurence [%]'
cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
cbar.set_label(label, fontsize=14)
cbar.ax.tick_params(labelsize=12)
plt.title('Nutrient limitation', fontsize=16)

ax2      = fig.add_subplot(122)
data, elem_no_nan = pf.get_data(data_lim_trend,mesh,0)
contours = np.arange(-2, 2.01, .01)
m = Basemap(projection='nplaea',boundinglat=70,lon_0=0,resolution='l')
x, y = m(mesh.x2, mesh.y2)
plabels=[False,False,False,False]
mlabels=[False,False,False,False]    
m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
m.drawmapboundary(fill_color='0.7')
m.fillcontinents(color='.5',lake_color='.7')
im=plt.tricontourf(x, y, elem_no_nan, (data) , levels=contours, cmap=cmap.cm.balance, extend='both')
label = 'Nutrient limitation occurence trend [% yr$^{-1}$]'
cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
cbar.set_label(label, fontsize=14)
cbar.ax.tick_params(labelsize=12)
plt.title('Nutrient limitation trend', fontsize=16)

plt.savefig('FIG3_map.png', dpi = 300, bbox_inches='tight') 

In [6]:
np.arange(-30,35,5)

array([-30, -25, -20, -15, -10,  -5,   0,   5,  10,  15,  20,  25,  30])

In [156]:
# data_lp_mean = data_lp.resample(time='YS').mean(dim='time')
# data_ld_mean = data_ld.resample(time='YS').mean(dim='time')

# data_lim_sertemp = 100 - 100*(data_lp_mean + data_ld_mean)/2
# data_lim_mean = data_lim_sertemp.mean(dim='time')
# pff = np.polyfit(years, data_lim_sertemp, 1)
# data_lim_trend = pff[0]

In [4]:
# fig = plt.figure(figsize=(15, 15), facecolor='w', edgecolor='k')

# #MLD_sept_mean[MLD_sept_mean == 0] = np.nan
# data, elem_no_nan = pf.get_data(data_lim_mean,mesh,0)
# ax1      = fig.add_subplot(111)
# contours = np.arange(50, 101, 1)
# m = Basemap(projection='nplaea',boundinglat=70,lon_0=0,resolution='l')
# x, y = m(mesh.x2, mesh.y2)
# plabels=[False,False,False,False]
# mlabels=[False,False,False,False]    
# m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
# m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
# m.drawmapboundary(fill_color='0.7')
# m.fillcontinents(color='.5',lake_color='.7')
# im=plt.tricontourf(x, y, elem_no_nan, (data) , levels=contours, cmap=cmap.cm.thermal, extend='min')
# label = 'Light limitation occurence [%]'
# cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
# cbar.set_label(label, fontsize=14)
# cbar.ax.tick_params(labelsize=12)
# plt.title('Light limitation occurence [%]', fontsize=16)

In [5]:
# fig = plt.figure(figsize=(15, 15), facecolor='w', edgecolor='k')

# #MLD_sept_mean[MLD_sept_mean == 0] = np.nan
# data, elem_no_nan = pf.get_data(data_lim_trend,mesh,0)
# ax1      = fig.add_subplot(111)
# contours = np.arange(-2, 2.01, .01)
# m = Basemap(projection='nplaea',boundinglat=70,lon_0=0,resolution='l')
# x, y = m(mesh.x2, mesh.y2)
# plabels=[False,False,False,False]
# mlabels=[False,False,False,False]    
# m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
# m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
# m.drawmapboundary(fill_color='0.7')
# m.fillcontinents(color='.5',lake_color='.7')
# im=plt.tricontourf(x, y, elem_no_nan, (data) , levels=contours, cmap=cmap.cm.balance, extend='both')
# label = 'Light limitation occurence trend [% yr$^{-1}$]'
# cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
# cbar.set_label(label, fontsize=14)
# cbar.ax.tick_params(labelsize=12)
# plt.title('Light limitation occurence trend [% yr$^{-1}$]', fontsize=16)